In [162]:
import pandas as pd
import numpy as np
import itertools
import plotly.express as px
import plotly.graph_objects as go
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

# Import & clean Adidas sales data

In [168]:
#Import sales data
adidas_sales_df = pd.read_csv('../adidas_sales.csv')

In [169]:
adidas_sales_df.head()

,Retailer_ID,Invoice_Date,Region_ID,State_ID,Product_ID,Price_per_unit,Units_sold,Operating_margin,Sales_method,Total_sales,Operating_profit
0,RTL1,2020-01-01,RG1,ST1,PD1,50.0,1200,0.50,In-store,60000.0,30000.0
1,RTL1,2020-01-02,RG1,ST1,PD2,50.0,1000,0.30,In-store,50000.0,15000.0
2,RTL1,2020-01-03,RG1,ST1,PD3,40.0,1000,0.35,In-store,40000.0,14000.0
3,RTL1,2020-01-04,RG1,ST1,PD4,45.0,850,0.35,In-store,38250.0,13387.5
4,RTL1,2020-01-05,RG1,ST1,PD5,60.0,900,0.30,In-store,54000.0,16200.0


In [170]:
#Check the data types
adidas_sales_df.dtypes

Retailer_ID          object
Invoice_Date         object
Region_ID            object
State_ID             object
Product_ID           object
Price_per_unit      float64
Units_sold            int64
Operating_margin    float64
Sales_method         object
Total_sales         float64
Operating_profit    float64
dtype: object

In [171]:
#Change the data type of the date column
adidas_sales_df['Invoice_Date'] = pd.to_datetime(adidas_sales_df['Invoice_Date'])

#Drop Retailer ID, Region_ID, Price per Unit, Units Sold, Operating Margin columns
adidas_sales_df = adidas_sales_df.drop(columns=['Retailer_ID', 'Region_ID', 'Price_per_unit', 'Units_sold', 'Operating_margin'])

adidas_sales_df

,Invoice_Date,State_ID,Product_ID,Sales_method,Total_sales,Operating_profit
0,2020-01-01,ST1,PD1,In-store,60000.0,30000.00
1,2020-01-02,ST1,PD2,In-store,50000.0,15000.00
2,2020-01-03,ST1,PD3,In-store,40000.0,14000.00
3,2020-01-04,ST1,PD4,In-store,38250.0,13387.50
4,2020-01-05,ST1,PD5,In-store,54000.0,16200.00
...,...,...,...,...,...,...
9641,2021-01-24,ST50,PD3,Outlet,3465.0,935.55
9642,2021-01-24,ST50,PD4,Outlet,1683.0,471.24
9643,2021-01-24,ST50,PD5,Outlet,3200.0,896.00
9644,2021-01-24,ST50,PD6,Outlet,4305.0,1377.60


In [172]:
#Group data by the sum of Total Sales per week
total_sales_weekly = adidas_sales_df.groupby(pd.Grouper(key='Invoice_Date', freq='W')).sum()

#Group data by the sum of Total Sales per month
total_sales_monthly = adidas_sales_df.groupby(pd.Grouper(key='Invoice_Date', freq='M')).sum()

#Group data by the sum of Total Sales per quarter
total_sales_quarterly = adidas_sales_df.groupby(pd.Grouper(key='Invoice_Date', freq='Q')).sum()

# Import & clean demographic data

In [173]:
#Retrieved from: https://apps.bea.gov/regional/downloadzip.cfm
#Import demographic data
demo_df = pd.read_csv('SQINC1__ALL_AREAS_1948_2022.csv')

In [174]:
demo_df.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1948:Q1,1948:Q2,...,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,"""00000""",United States,,SQINC1,1.0,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,204641.7,210069.4,...,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,"""00000""",United States,,SQINC1,2.0,...,"Population (midperiod, persons) 1/",Number of persons,(NA),(NA),...,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,"""00000""",United States,,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0
3,"""01000""",Alabama,5,SQINC1,1.0,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,2496.0,2595.6,...,242348.1,232629.8,230152.7,265461.3,244059.8,245116.7,248677.4,250483.3,254302.5,257231.8
4,"""01000""",Alabama,5,SQINC1,2.0,...,"Population (midperiod, persons) 1/",Number of persons,(NA),(NA),...,5024115.0,5027375.0,5031760.0,5033508.0,5036858.0,5043548.0,5050555.0,5055254.0,5060373.0,5067413.0


In [175]:
#Drop the columns that are not needed
demo_df = demo_df.drop(columns=['GeoFIPS', 'Region', 'TableName', 'LineCode', 'IndustryClassification',])

demo_df.columns

Index(['GeoName', 'Description', 'Unit', '1948:Q1', '1948:Q2', '1948:Q3',
       '1948:Q4', '1949:Q1', '1949:Q2', '1949:Q3',
       ...
       '2020:Q2', '2020:Q3', '2020:Q4', '2021:Q1', '2021:Q2', '2021:Q3',
       '2021:Q4', '2022:Q1', '2022:Q2', '2022:Q3'],
      dtype='object', length=302)

In [176]:
#Drop column years that are not needed
demo_df = demo_df.drop(demo_df.columns[3:-11], axis=1)

#Drop NaN values
demo_df = demo_df.dropna()

demo_df

,GeoName,Description,Unit,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,United States,"Personal income (millions of dollars, seasonal...",Millions of dollars,19013184.9,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,United States,"Population (midperiod, persons) 1/",Number of persons,331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,United States,Per capita personal income (dollars) 2/,Dollars,57390.0,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0
3,Alabama,"Personal income (millions of dollars, seasonal...",Millions of dollars,223030.6,242348.1,232629.8,230152.7,265461.3,244059.8,245116.7,248677.4,250483.3,254302.5,257231.8
4,Alabama,"Population (midperiod, persons) 1/",Number of persons,5021627.0,5024115.0,5027375.0,5031760.0,5033508.0,5036858.0,5043548.0,5050555.0,5055254.0,5060373.0,5067413.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Rocky Mountain,"Population (midperiod, persons) 1/",Number of persons,12527279.0,12560325.0,12598245.0,12638757.0,12673366.0,12711764.0,12756878.0,12802158.0,12844282.0,12887858.0,12934628.0
176,Rocky Mountain,Per capita personal income (dollars) 2/,Dollars,56914.0,60464.0,58482.0,58654.0,65105.0,61482.0,61865.0,63032.0,63699.0,64483.0,65851.0
177,Far West,"Personal income (millions of dollars, seasonal...",Millions of dollars,3676803.4,3928385.4,3947530.0,3893315.9,4281973.3,4100396.5,4137883.4,4135892.3,4123489.4,4161472.7,4211194.0
178,Far West,"Population (midperiod, persons) 1/",Number of persons,56763245.0,56762231.0,56739149.0,56691552.0,56618257.0,56558476.0,56530151.0,56503693.0,56461835.0,56422276.0,56398026.0


In [177]:
demo_df['GeoName'].unique()

array(['United States', 'Alabama', 'Alaska *', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii *', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'New England', 'Mideast',
       'Great Lakes', 'Plains', 'Southeast', 'Southwest',
       'Rocky Mountain', 'Far West'], dtype=object)

In [178]:
#Remove the states from the GeoName column
demo_df = demo_df.loc[(demo_df['GeoName'] != 'Alabama') & (demo_df['GeoName'] != 'Alaska *') & (demo_df['GeoName'] != 'Arizona') & (demo_df['GeoName'] != 'Arkansas') & (demo_df['GeoName'] != 'California') & (demo_df['GeoName'] != 'Colorado') & (demo_df['GeoName'] != 'Connecticut') & (demo_df['GeoName'] != 'Delaware') & (demo_df['GeoName'] != 'District of Columbia') & (demo_df['GeoName'] != 'Florida') & (demo_df['GeoName'] != 'Georgia') & (demo_df['GeoName'] != 'Hawaii *') & (demo_df['GeoName'] != 'Idaho') & (demo_df['GeoName'] != 'Illinois') & (demo_df['GeoName'] != 'Indiana') & (demo_df['GeoName'] != 'Iowa') & (demo_df['GeoName'] != 'Kansas') & (demo_df['GeoName'] != 'Kentucky') & (demo_df['GeoName'] != 'Louisiana') & (demo_df['GeoName'] != 'Maine') & (demo_df['GeoName'] != 'Maryland') & (demo_df['GeoName'] != 'Massachusetts') & (demo_df['GeoName'] != 'Michigan') & (demo_df['GeoName'] != 'Minnesota') & (demo_df['GeoName'] != 'Mississippi') & (demo_df['GeoName'] != 'Missouri') & (demo_df['GeoName'] != 'Montana') & (demo_df['GeoName'] != 'Nebraska') & (demo_df['GeoName'] != 'Nevada') & (demo_df['GeoName'] != 'New Hampshire') & (demo_df['GeoName'] != 'New Jersey') & (demo_df['GeoName'] != 'New Mexico') & (demo_df['GeoName'] != 'New York') & (demo_df['GeoName'] != 'North Carolina') & (demo_df['GeoName'] != 'North Dakota') & (demo_df['GeoName'] != 'Ohio') & (demo_df['GeoName'] != 'Oklahoma') & (demo_df['GeoName'] != 'Oregon') & (demo_df['GeoName'] != 'Pennsylvania') & (demo_df['GeoName'] != 'Rhode Island') & (demo_df['GeoName'] != 'South Carolina') & (demo_df['GeoName'] != 'South Dakota') & (demo_df['GeoName'] != 'Tennessee') & (demo_df['GeoName'] != 'Texas') & (demo_df['GeoName'] != 'Utah') & (demo_df['GeoName'] != 'Vermont') & (demo_df['GeoName'] != 'Virginia') & (demo_df['GeoName'] != 'Washington') & (demo_df['GeoName'] != 'West Virginia') & (demo_df['GeoName'] != 'Wisconsin') & (demo_df['GeoName'] != 'Wyoming')]

#Remove the regions from the GeoName column
demo_df = demo_df.loc[(demo_df['GeoName'] != 'New England') & (demo_df['GeoName'] != 'Mideast') & (demo_df['GeoName'] != 'Great Lakes') & (demo_df['GeoName'] != 'Plains') & (demo_df['GeoName'] != 'Southeast') & (demo_df['GeoName'] != 'Southwest') & (demo_df['GeoName'] != 'Rocky Mountain') & (demo_df['GeoName'] != 'Far West')]

demo_df

,GeoName,Description,Unit,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,United States,"Personal income (millions of dollars, seasonal...",Millions of dollars,19013184.9,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,United States,"Population (midperiod, persons) 1/",Number of persons,331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,United States,Per capita personal income (dollars) 2/,Dollars,57390.0,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0


In [179]:
#Remove GeoName and Unit columns
demo_df = demo_df.drop(columns=['GeoName', 'Unit'])

#Clean Description column entries
demo_df['Description'] = demo_df['Description'].str.replace(' 1/', '')
demo_df['Description'] = demo_df['Description'].str.replace(' 2/', '')

demo_df

,Description,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,"Personal income (millions of dollars, seasonal...",19013184.9,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,"Population (midperiod, persons)",331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,Per capita personal income (dollars),57390.0,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0


# Merge Adidas sales data with demographic data

In [180]:
#Set the Description column as the index to avoid using it as a column
merged_df = demo_df.set_index('Description')

In [181]:
#Transpose sales data to match the demographic data
total_sales_quarterly = total_sales_quarterly.transpose()

In [182]:
#Add empty sales data to match the demographic data timeline
total_sales_quarterly['2022:Q1'] = np.nan
total_sales_quarterly['2022:Q2'] = np.nan
total_sales_quarterly['2022:Q3'] = np.nan

#Rename total_sales_quarterly columns
total_sales_quarterly.columns = ['2020:Q1', '2020:Q2', '2020:Q3', '2020:Q4', '2021:Q1', '2021:Q2', '2021:Q3', '2021:Q4', '2022:Q1', '2022:Q2', '2022:Q3']

In [183]:
merged_df = pd.concat([merged_df, total_sales_quarterly], axis=0)
merged_df

,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
"Personal income (millions of dollars, seasonally adjusted)",1.901318e+07,2.045938e+07,1.999781e+07,1.977832e+07,2.209004e+07,2.090786e+07,2.099890e+07,2.115804e+07,21317801.6,21575362.1,21856480.2
"Population (midperiod, persons)",3.312959e+08,3.314482e+08,3.315966e+08,3.317343e+08,3.317063e+08,3.317762e+08,3.320500e+08,3.323368e+08,332502197.0,332693300.0,332994420.0
Per capita personal income (dollars),5.739000e+04,6.172700e+04,6.030800e+04,5.962100e+04,6.659500e+04,6.301800e+04,6.324000e+04,6.366400e+04,64113.0,64851.0,65636.0
Total_sales,6.927761e+06,6.442039e+06,7.191704e+06,3.675821e+06,1.877050e+07,2.379425e+07,2.805752e+07,2.530171e+07,NaN,NaN,NaN
Operating_profit,2.559891e+06,2.383124e+06,2.700035e+06,1.372455e+06,7.113925e+06,9.530545e+06,1.139524e+07,1.016787e+07,NaN,NaN,NaN


# Predictive analysis with multiple linear regression

In [184]:
#Today I learned that train_test_split only works with columns rather than rows
merged_df = merged_df.transpose()

In [185]:
#Define feature set (which includes all three demographic columns)
X_quarterly = merged_df.iloc[:-3, :-2]

#Define target set (which includes the total sales and operating profit columns)
y_quarterly = merged_df.iloc[:-3, 3:]

In [186]:
#Split data into training and testing sets
X_quarterly_train, X_quarterly_test, y_quarterly_train, y_quarterly_test = train_test_split(X_quarterly, y_quarterly, random_state=31)

## Random Forest Regressor

In [187]:
#Create a random forest regressor
rfr = RandomForestRegressor(n_estimators=100, random_state=0)

#Fit the model to the training data
rfr.fit(X_quarterly_train, y_quarterly_train)

#Make predictions using the testing data
rfr_predictions = rfr.predict(X_quarterly_test)

In [188]:
#Calculate the mean squared error and root mean squared error as performance metrics
rfr_mse = mean_squared_error(y_quarterly_test, rfr_predictions)
rfr_rmse = rfr_mse**.5

print(f'Random Forest Regressor MSE: {rfr_mse}')
print(f'Random Forest Regressor RMSE: {rfr_rmse}')

Random Forest Regressor MSE: 14875752818851.555
Random Forest Regressor RMSE: 3856909.7498971317


In [189]:
#A RMSE of 3856909 means that the model is off by about $3.85 million on average

#Create a dataframe of the predictions
rfr_df = pd.DataFrame(rfr_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_quarterly_test.index)

#Add the actual values to the dataframe
rfr_df['Actual_Total_Sales'], rfr_df['Actual_Operating_Profit'] = y_quarterly_test['Total_sales'], y_quarterly_test['Operating_profit']

#Sort index in order to match the timeline
rfr_df.sort_index(inplace=True)

rfr_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q2,13465296.72,5.291072e+06,6442039.0,2383123.95
2021:Q1,19789758.82,7.939638e+06,18770496.0,7113924.66


In [190]:
#Plot the predictions vs. the actual values
px.line(rfr_df, x=rfr_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Random Forest Regressor: Total Sales & Operating Profit Predictions vs. Actual Values (Quarterly Data)')

In [191]:
#Define the scoring function to be used in GridSearchCV
mse = make_scorer(mean_squared_error, greater_is_better=False)

#Find optimal parameters using GridSearchCV
rfr_grid = {'n_estimators': [50, 100, 200, 300, 400], 
        'max_depth': [None, 5, 10, 15, 20], 
        'min_samples_split': [2, 4, 6, 8, 10], 
        'min_samples_leaf': [1, 2, 3, 4, 5], 
        'max_features': ['auto', 'sqrt', 'log2']}

#Grid search function
CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=rfr_grid, scoring=mse, n_jobs=4)
CV_rfr.fit(X_quarterly_train, y_quarterly_train)

GridSearchCV(estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'n_estimators': [50, 100, 200, 300, 400]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [192]:
#Show the best parameters
CV_rfr.best_params_

{'max_depth': None,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [193]:
#Create a random forest regressor
opt_rfr = RandomForestRegressor(max_depth=None, max_features= 'log2', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 100, random_state=0)

#Fit the model to the training data
opt_rfr.fit(X_quarterly_train, y_quarterly_train)

#Make predictions using the testing data
opt_rfr_predictions = opt_rfr.predict(X_quarterly_test)

In [194]:
#Calculate the mean squared error and root mean squared error as performance metrics
opt_rfr_mse = mean_squared_error(y_quarterly_test, opt_rfr_predictions)
opt_rfr_rmse = opt_rfr_mse**.5

print(f'Random Forest Regressor MSE: {opt_rfr_mse}')
print(f'Random Forest Regressor RMSE: {opt_rfr_rmse}')

Random Forest Regressor MSE: 14904899180216.934
Random Forest Regressor RMSE: 3860686.361285637


In [223]:
#After doing some manual iterations of the parameters, it seems that GridSearchCV was not able to produce the best results
#This could possibly be due to the very small dataset. Somehow adding a defined scoring function to GridSearchCV made the RMSE worse

opt_rfr_rmse_loop = 3860686

for n in rfr_grid['n_estimators']:
    for d in rfr_grid['max_depth']:
        for s in rfr_grid['min_samples_split']:
            for l in rfr_grid['min_samples_leaf']:
                for f in rfr_grid['max_features']:
                    opt_rfr = RandomForestRegressor(max_depth=d, max_features=f, min_samples_leaf=l, min_samples_split=s, n_estimators=n, random_state=0)
                    opt_rfr.fit(X_quarterly_train, y_quarterly_train)
                    opt_rfr_predictions = opt_rfr.predict(X_quarterly_test)
                    opt_rfr_rmse_in_loop = mean_squared_error(y_quarterly_test, opt_rfr_predictions)**.5
                    
                    #If the RMSE is lower than the previous RMSE, then update
                    if opt_rfr_rmse_in_loop < opt_rfr_rmse_loop:
                        opt_rfr_rmse_loop = opt_rfr_rmse_in_loop
                        print(f'RMSE: {opt_rfr_rmse_loop}')
                        print(f'Optimal parameters: n_estimators={n}, max_depth={d}, min_samples_split={s}, min_samples_leaf={l}, max_features={f}')
                    

RMSE: 3777590.1141415015
Optimal parameters: n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=auto
RMSE: 3708158.6286961916
Optimal parameters: n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=sqrt


In [196]:
#Create a random forest regressor
opt_rfr = RandomForestRegressor(max_depth=None, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 50, random_state=0)

#Fit the model to the training data
opt_rfr.fit(X_quarterly_train, y_quarterly_train)

#Make predictions using the testing data
opt_rfr_predictions = opt_rfr.predict(X_quarterly_test)

In [197]:
#Calculate the mean squared error and root mean squared error as performance metrics
opt_rfr_mse = mean_squared_error(y_quarterly_test, opt_rfr_predictions)
opt_rfr_rmse = opt_rfr_mse**.5

print(f'Random Forest Regressor MSE: {opt_rfr_mse}')
print(f'Random Forest Regressor RMSE: {opt_rfr_rmse}')

Random Forest Regressor MSE: 13750440415574.021
Random Forest Regressor RMSE: 3708158.6286961916


In [198]:
#Create a dataframe of the predictions
opt_rfr_df = pd.DataFrame(opt_rfr_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_quarterly_test.index)

#Add the actual values to the dataframe
opt_rfr_df['Actual_Total_Sales'], opt_rfr_df['Actual_Operating_Profit'] = y_quarterly_test['Total_sales'], y_quarterly_test['Operating_profit']

#Sort index in order to match the timeline
opt_rfr_df.sort_index(inplace=True)

opt_rfr_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q2,13270180.32,5.219826e+06,6442039.0,2383123.95
2021:Q1,19043236.04,7.620859e+06,18770496.0,7113924.66


In [199]:
#Plot the predictions vs. the actual values
px.line(opt_rfr_df, x=opt_rfr_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Optimized Random Forest Regressor: Total Sales & Operating Profit Predictions vs. Actual Values (Quarterly Data)')

In [200]:
#Create a dataframe of the predictions
opt_rfr_df = pd.DataFrame(test2, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=test.index)

#Add the actual values to the dataframe
opt_rfr_df['Actual_Total_Sales'], opt_rfr_df['Actual_Operating_Profit'] = merged_df['Total_sales'], merged_df['Operating_profit']

#Sort index in order to match the timeline
opt_rfr_df.sort_index(inplace=True)

opt_rfr_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q1,6852050.02,2.555486e+06,6927761.0,2559891.11
2020:Q2,13270180.32,5.219826e+06,6442039.0,2383123.95
2020:Q3,7414362.38,2.817798e+06,7191704.0,2700034.59
2020:Q4,6450600.24,2.474726e+06,3675821.0,1372455.43
2021:Q1,19043236.04,7.620859e+06,18770496.0,7113924.66
2021:Q2,21754829.74,8.710408e+06,23794248.0,9530544.61
2021:Q3,26733789.82,1.081317e+07,28057525.0,11395237.28
2021:Q4,25872657.18,1.042420e+07,25301709.0,10167871.38
2022:Q1,25872657.18,1.042420e+07,NaN,NaN
2022:Q2,25872657.18,1.042420e+07,NaN,NaN


In [201]:
#Plot the predictions vs. the actual values
px.line(opt_rfr_df, x=opt_rfr_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Optimized Random Forest Regressor: Total Sales & Operating Profit Predictions vs. Actual Values (Quarterly Data)')

## Split data by month rather than quarter

In [202]:
monthly_merged_df = demo_df.set_index('Description')

In [203]:
total_sales_monthly = total_sales_monthly.transpose()
total_sales_monthly

Invoice_Date,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,...,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31
Total_sales,2312746.00,2140813.00,2474202.00,3193081.00,2164764.0,1084194.00,2182388.00,2641630.00,2367686.00,1428569.00,...,5220782.00,6498339.00,8576956.00,8718953.00,10368031.00,9651596.00,8037898.00,7110189.00,7855390.00,10336130.0
Operating_profit,883774.16,796848.19,879268.76,1275442.09,800635.7,307046.16,681477.61,1042648.29,975908.69,582342.96,...,2067129.24,2644237.72,3456570.88,3429736.01,4098805.98,3852859.51,3443571.79,2987755.39,3101992.69,4078123.3


In [204]:
#Add empty columns to match the shape of demo_df
total_sales_monthly['2022-01-31'] = np.nan
total_sales_monthly['2022-02-29'] = np.nan
total_sales_monthly['2022-03-31'] = np.nan

total_sales_monthly['2022-04-30'] = np.nan
total_sales_monthly['2022-05-31'] = np.nan
total_sales_monthly['2022-06-30'] = np.nan

total_sales_monthly['2022-07-31'] = np.nan
total_sales_monthly['2022-08-31'] = np.nan
total_sales_monthly['2022-09-30'] = np.nan

In [205]:
#Set the Description column as the index to avoid using it as a column
demo_df_elongated = demo_df.set_index('Description')

In [206]:
#Rename each column to three months per quarter, ie. 2020:Q1, 2020:Q1_2, 2020:Q1_3
quarter_x3 = [2, 3]

for column in demo_df_elongated.columns:
    for quarter in quarter_x3:
            demo_df_elongated[f'{column}_{quarter}'] = demo_df_elongated[column]

In [207]:
#Sort columns
demo_df_elongated = demo_df_elongated.reindex(sorted(demo_df_elongated.columns), axis=1)

In [208]:
demo_df_elongated

,2020:Q1,2020:Q1_2,2020:Q1_3,2020:Q2,2020:Q2_2,2020:Q2_3,2020:Q3,2020:Q3_2,2020:Q3_3,2020:Q4,...,2021:Q4_3,2022:Q1,2022:Q1_2,2022:Q1_3,2022:Q2,2022:Q2_2,2022:Q2_3,2022:Q3,2022:Q3_2,2022:Q3_3
Description,,,,,,,,,,,,,,,,,,,,,
"Personal income (millions of dollars, seasonally adjusted)",19013184.9,19013184.9,19013184.9,20459375.8,20459375.8,20459375.8,19997807.5,19997807.5,19997807.5,19778315.9,...,21158043.8,21317801.6,21317801.6,21317801.6,21575362.1,21575362.1,21575362.1,21856480.2,21856480.2,21856480.2
"Population (midperiod, persons)",331295939.0,331295939.0,331295939.0,331448217.0,331448217.0,331448217.0,331596557.0,331596557.0,331596557.0,331734262.0,...,332336782.0,332502197.0,332502197.0,332502197.0,332693300.0,332693300.0,332693300.0,332994420.0,332994420.0,332994420.0
Per capita personal income (dollars),57390.0,57390.0,57390.0,61727.0,61727.0,61727.0,60308.0,60308.0,60308.0,59621.0,...,63664.0,64113.0,64113.0,64113.0,64851.0,64851.0,64851.0,65636.0,65636.0,65636.0


In [209]:
#Match column names
total_sales_monthly.columns = demo_df_elongated.columns

total_sales_monthly

,2020:Q1,2020:Q1_2,2020:Q1_3,2020:Q2,2020:Q2_2,2020:Q2_3,2020:Q3,2020:Q3_2,2020:Q3_3,2020:Q4,...,2021:Q4_3,2022:Q1,2022:Q1_2,2022:Q1_3,2022:Q2,2022:Q2_2,2022:Q2_3,2022:Q3,2022:Q3_2,2022:Q3_3
Total_sales,2312746.00,2140813.00,2474202.00,3193081.00,2164764.0,1084194.00,2182388.00,2641630.00,2367686.00,1428569.00,...,10336130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Operating_profit,883774.16,796848.19,879268.76,1275442.09,800635.7,307046.16,681477.61,1042648.29,975908.69,582342.96,...,4078123.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
#Merge the demographic data with the total sales data
monthly_merged_df = pd.concat([demo_df_elongated, total_sales_monthly], axis=0)
monthly_merged_df

,2020:Q1,2020:Q1_2,2020:Q1_3,2020:Q2,2020:Q2_2,2020:Q2_3,2020:Q3,2020:Q3_2,2020:Q3_3,2020:Q4,...,2021:Q4_3,2022:Q1,2022:Q1_2,2022:Q1_3,2022:Q2,2022:Q2_2,2022:Q2_3,2022:Q3,2022:Q3_2,2022:Q3_3
"Personal income (millions of dollars, seasonally adjusted)",1.901318e+07,1.901318e+07,1.901318e+07,2.045938e+07,20459375.8,2.045938e+07,1.999781e+07,1.999781e+07,1.999781e+07,1.977832e+07,...,21158043.8,21317801.6,21317801.6,21317801.6,21575362.1,21575362.1,21575362.1,21856480.2,21856480.2,21856480.2
"Population (midperiod, persons)",3.312959e+08,3.312959e+08,3.312959e+08,3.314482e+08,331448217.0,3.314482e+08,3.315966e+08,3.315966e+08,3.315966e+08,3.317343e+08,...,332336782.0,332502197.0,332502197.0,332502197.0,332693300.0,332693300.0,332693300.0,332994420.0,332994420.0,332994420.0
Per capita personal income (dollars),5.739000e+04,5.739000e+04,5.739000e+04,6.172700e+04,61727.0,6.172700e+04,6.030800e+04,6.030800e+04,6.030800e+04,5.962100e+04,...,63664.0,64113.0,64113.0,64113.0,64851.0,64851.0,64851.0,65636.0,65636.0,65636.0
Total_sales,2.312746e+06,2.140813e+06,2.474202e+06,3.193081e+06,2164764.0,1.084194e+06,2.182388e+06,2.641630e+06,2.367686e+06,1.428569e+06,...,10336130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Operating_profit,8.837742e+05,7.968482e+05,8.792688e+05,1.275442e+06,800635.7,3.070462e+05,6.814776e+05,1.042648e+06,9.759087e+05,5.823430e+05,...,4078123.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Random Forest Regressor with month split

In [211]:
#Transpose the dataframe to fit shape for train_test_split
monthly_merged_df = monthly_merged_df.transpose()

In [212]:
#Define feature set (which includes all three demographic columns)
X = monthly_merged_df.iloc[:-9, :3]

#Define target set (which includes the total sales and operating profit columns)
y = monthly_merged_df.iloc[:-9, 3:]

#Prediction sets (which includes columns where total sales and operating profit are NaN)
X_pred = monthly_merged_df.iloc[-9:, :3]
y_pred = monthly_merged_df.iloc[-9:, 3:]

In [213]:
#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=31)

In [214]:
#Create a random forest regressor
rfr = RandomForestRegressor(n_estimators=100, random_state=0)

#Fit the model to the training data
rfr.fit(X_train, y_train)

#Make predictions using the testing data
rfr_predictions = rfr.predict(X_test)

In [215]:
#Calculate the mean squared error and root mean squared error as performance metrics
rfr_mse = mean_squared_error(y_test, rfr_predictions)
rfr_rmse = rfr_mse**.5

print(f'Random Forest Regressor MSE: {rfr_mse}')
print(f'Random Forest Regressor RMSE: {rfr_rmse}')

Random Forest Regressor MSE: 824943196061.6461
Random Forest Regressor RMSE: 908263.8361520545


In [216]:
#A RMSE of 908263 means that the model is off by about $908,263 on average

#Create a dataframe of the predictions
rfr_df = pd.DataFrame(rfr_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_test.index)
rfr_df['Actual_Total_Sales'], rfr_df['Actual_Operating_Profit'] = y_test['Total_sales'], y_test['Operating_profit']
rfr_df = rfr_df.sort_index()
rfr_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q1,2.215932e+06,8.043960e+05,2312746.0,883774.16
2020:Q2_3,3.066253e+06,1.182672e+06,1084194.0,307046.16
2020:Q3_2,2.300622e+06,8.526891e+05,2641630.0,1042648.29
2020:Q4,1.603434e+06,5.879956e+05,1428569.0,582342.96
2021:Q2,8.395391e+06,3.343996e+06,6498339.0,2644237.72
2021:Q4_2,8.708044e+06,3.537770e+06,7855390.0,3101992.69


In [217]:
#Plot the predictions vs. the actual values
px.line(rfr_df, x=rfr_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Random Forest Regressor: Total Sales & Operating Profit Predictions vs. Actual Values (Monthly Data)')

In [218]:
#Define the scoring function to be used in GridSearchCV
mse = make_scorer(mean_squared_error, greater_is_better=False)

#Find optimal parameters using GridSearchCV
opt_rfr_grid = {'n_estimators': [50, 100, 200, 300, 400], 
        'max_depth': [None, 5, 10, 15, 20], 
        'min_samples_split': [2, 4, 6, 8, 10], 
        'min_samples_leaf': [1, 2, 3, 4, 5], 
        'max_features': ['auto', 'sqrt', 'log2']}

#Grid search function
CV_opt_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=opt_rfr_grid, scoring=mse, n_jobs=4)
CV_opt_rfr.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'n_estimators': [50, 100, 200, 300, 400]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [219]:
#Show the best parameters
CV_opt_rfr.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 50}

In [220]:
#Create a random forest regressor
opt_rfr = RandomForestRegressor(max_depth=None, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split= 2, n_estimators= 50, random_state=0)

#Fit the model to the training data
opt_rfr.fit(X_train, y_train)

#Make predictions using the testing data
opt_rfr_predictions = opt_rfr.predict(X_test)

In [221]:
#Calculate the mean squared error and root mean squared error as performance metrics
opt_rfr_mse = mean_squared_error(y_test, opt_rfr_predictions)
opt_rfr_rmse = opt_rfr_mse**.5

print(f'Random Forest Regressor MSE: {opt_rfr_mse}')
print(f'Random Forest Regressor RMSE: {opt_rfr_rmse}')

Random Forest Regressor MSE: 679252844329.5784
Random Forest Regressor RMSE: 824167.9709437746


In [224]:
#For loop to find the optimal parameters
opt_rfr_rmse_loop = 824167

for n in opt_rfr_grid['n_estimators']:
    for d in opt_rfr_grid['max_depth']:
        for s in opt_rfr_grid['min_samples_split']:
            for l in opt_rfr_grid['min_samples_leaf']:
                for f in opt_rfr_grid['max_features']:
                    opt_rfr = RandomForestRegressor(max_depth=d, max_features=f, min_samples_leaf=l, min_samples_split=s, n_estimators=n, random_state=0)
                    opt_rfr.fit(X_train, y_train)
                    opt_rfr_predictions = opt_rfr.predict(X_test)
                    opt_rfr_rmse_in_loop = mean_squared_error(y_test, opt_rfr_predictions)**.5
                    
                    #If the RMSE is lower than the previous RMSE, then update
                    if opt_rfr_rmse_in_loop < opt_rfr_rmse_loop:
                        opt_rfr_rmse_loop = opt_rfr_rmse_in_loop
                        print(f'RMSE: {opt_rfr_rmse_loop}')
                        print(f'Optimal parameters: n_estimators={n}, max_depth={d}, min_samples_split={s}, min_samples_leaf={l}, max_features={f}')
                    

RMSE: 810141.2761063573
Optimal parameters: n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=sqrt
RMSE: 807576.2527010938
Optimal parameters: n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=4, max_features=auto
RMSE: 769475.8856729567
Optimal parameters: n_estimators=50, max_depth=None, min_samples_split=4, min_samples_leaf=1, max_features=sqrt


In [228]:
#Create a random forest regressor
opt_rfr = RandomForestRegressor(max_depth=None, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 4, n_estimators= 50, random_state=0)

#Fit the model to the training data
opt_rfr.fit(X_train, y_train)

#Make predictions using the testing data
opt_rfr_predictions = opt_rfr.predict(X_test)

In [229]:
#A RMSE of 3708158 means that the model is off by about $3.7 million on average

#Create a dataframe of the predictions
opt_rfr_df = pd.DataFrame(opt_rfr_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_test.index)
opt_rfr_df['Actual_Total_Sales'], opt_rfr_df['Actual_Operating_Profit'] = y_test['Total_sales'], y_test['Operating_profit']
opt_rfr_df = opt_rfr_df.sort_index()
opt_rfr_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q1,2.038694e+06,7.402727e+05,2312746.0,883774.16
2020:Q2_3,2.828291e+06,1.092221e+06,1084194.0,307046.16
2020:Q3_2,2.183755e+06,8.078539e+05,2641630.0,1042648.29
2020:Q4,1.778379e+06,6.435760e+05,1428569.0,582342.96
2021:Q2,7.905664e+06,3.160981e+06,6498339.0,2644237.72
2021:Q4_2,8.604739e+06,3.490633e+06,7855390.0,3101992.69


In [230]:
#Plot the predictions vs. the actual values
px.line(opt_rfr_df, x=opt_rfr_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Optimized Random Forest Regressor: Total Sales & Operating Profit Predictions vs. Actual Values (Monthly Data)')

In [231]:
#Make predictions using the testing data
opt_rfr_predictions_future = opt_rfr.predict(X_pred)

In [232]:
#Plot total_sales_monthly and opt_rfr_predictions_future
px.line(monthly_merged_df, x=monthly_merged_df.index, y=['Total_sales', 'Operating_profit'], title='Actual Sales With Predicted Sales')\
    .add_scatter(x=monthly_merged_df.iloc[-9:].index, y=opt_rfr_predictions_future[:, 0], mode='lines', name='Total Sales Predictions')\
    .add_scatter(x=monthly_merged_df.iloc[-9:].index, y=opt_rfr_predictions_future[:, 1], mode='lines', name='Operating Profit Predictions')

## Support Vector Regression

In [236]:
#Create model
svr = SVR(kernel='linear', C=100, gamma=0.1, epsilon=.1)

#Fit model to training data. SVR only accepts 1D arrays, so we need to drop the Operating_profit column
svr.fit(X_train, y_train.drop('Operating_profit', axis=1).values.ravel())

#Make predictions using the testing data
svr_predictions = svr.predict(X_test)

In [243]:
#Calculate the mean squared error and root mean squared error as performance metrics
svr_mse = mean_squared_error(y_test.drop('Operating_profit', axis=1).values.ravel(), svr_predictions)
svr_rmse = svr_mse**.5

print(f'Random Forest Regressor MSE: {svr_mse}')
print(f'Random Forest Regressor RMSE: {svr_rmse}')

Random Forest Regressor MSE: 10815879846761.916
Random Forest Regressor RMSE: 3288750.499317623


In [239]:
#A RMSE of 3288750 means that the model is off by about $3,288,750 on average

#Create a dataframe of the predictions
svr_df = pd.DataFrame(svr_predictions, columns=['Predicted_Total_Sales'], index=y_test.index).sort_index()

#Add the actual values to the dataframe
svr_df['Actual_Total_Sales'] = y_test['Total_sales']

svr_df

,Predicted_Total_Sales,Actual_Total_Sales
2020:Q1,5668937.5,2312746.0
2020:Q2_3,5668937.5,1084194.0
2020:Q3_2,5668937.5,2641630.0
2020:Q4,5668937.5,1428569.0
2021:Q2,5669337.5,6498339.0
2021:Q4_2,5669337.5,7855390.0


In [240]:
#plot the predictions vs. the actual values
px.line(svr_df, x=svr_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales'], 
        title='Support Vector Regression: Total Sales Predictions vs. Actual Values (Monthly Data)')

In [245]:
#Find the optimal parameters for the SVR model
mse = make_scorer(mean_squared_error, greater_is_better=False)

#
svr_grid = {'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [0.1, 0.01, 0.001, 0.0001], 'epsilon': [.1, .01, .001, .0001]}

#Create a grid search object
CV_opt_svr = GridSearchCV(estimator=SVR(), param_grid=svr_grid, scoring=mse, cv=5)
CV_opt_svr.fit(X_train, y_train.drop('Operating_profit', axis=1).values.ravel())

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 10, 100, 1000],
                         'epsilon': [0.1, 0.01, 0.001, 0.0001],
                         'gamma': [0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [246]:
#Show the best parameters
CV_opt_svr.best_params_

{'C': 1000, 'epsilon': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}

In [247]:
#Create model
opt_svr = SVR(kernel='rbf', C=1000, gamma=0.1, epsilon=.1)

#Fit model to training data. SVR only accepts 1D arrays, so we need to drop the Operating_profit column
opt_svr.fit(X_train, y_train.drop('Operating_profit', axis=1).values.ravel())

#Make predictions using the testing data
opt_svr_predictions = opt_svr.predict(X_test)

In [248]:
#Calculate the mean squared error and root mean squared error as performance metrics
opt_svr_mse = mean_squared_error(y_test.drop('Operating_profit', axis=1).values.ravel(), opt_svr_predictions)
opt_svr_rmse = opt_svr_mse**.5

print(f'Random Forest Regressor MSE: {opt_svr_mse}')
print(f'Random Forest Regressor RMSE: {opt_svr_rmse}')

Random Forest Regressor MSE: 10801292710661.916
Random Forest Regressor RMSE: 3286532.018809784


In [249]:
#A RMSE of 3288750 means that the model is off by about $3,288,750 on average

#Create a dataframe of the predictions
opt_svr_df = pd.DataFrame(opt_svr_predictions, columns=['Predicted_Total_Sales'], index=y_test.index).sort_index()

#Add the actual values to the dataframe
opt_svr_df['Actual_Total_Sales'] = y_test['Total_sales']

opt_svr_df

,Predicted_Total_Sales,Actual_Total_Sales
2020:Q1,5666237.5,2312746.0
2020:Q2_3,5666237.5,1084194.0
2020:Q3_2,5666237.5,2641630.0
2020:Q4,5666237.5,1428569.0
2021:Q2,5670237.5,6498339.0
2021:Q4_2,5670237.5,7855390.0


In [250]:
#plot the predictions vs. the actual values
px.line(opt_svr_df, x=opt_svr_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales'], 
        title='Optimized Support Vector Regression: Total Sales Predictions vs. Actual Values (Monthly Data)')

## Decision Tree Regression

In [251]:
#Create model
dtr_model = DecisionTreeRegressor(max_depth=5, random_state=0)

#Fit model to training data
dtr_model.fit(X_train, y_train)

#Make predictions using the testing data
dtr_predictions = dtr_model.predict(X_test)

In [252]:
#Accuracy of the model
dtr_mse = mean_squared_error(y_test, dtr_predictions)
dtr_rmse = dtr_mse**.5

print(f'Random Forest Regressor MSE: {dtr_mse}')
print(f'Random Forest Regressor RMSE: {dtr_rmse}')

Random Forest Regressor MSE: 798809380945.2006
Random Forest Regressor RMSE: 893761.3668900668


In [254]:
#Create a dataframe of the predictions
dtr_predictions_df = pd.DataFrame(dtr_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_test.index)
dtr_predictions_df['Actual_Total_Sales'], dtr_predictions_df['Actual_Operating_Profit'] = y_test['Total_sales'], y_test['Operating_profit']
dtr_predictions_df = dtr_predictions_df.sort_index()
dtr_predictions_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q1,2307507.5,838058.475,2312746.0,883774.16
2020:Q2_3,2678922.5,1038038.895,1084194.0,307046.16
2020:Q3_2,2275037.0,828693.150,2641630.0,1042648.29
2020:Q4,1123626.0,395056.235,1428569.0,582342.96
2021:Q2,8647954.5,3443153.445,6498339.0,2644237.72
2021:Q4_2,8723159.5,3532939.345,7855390.0,3101992.69


In [255]:
#Plot the predictions vs. the actual values
px.line(dtr_predictions_df, x=dtr_predictions_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Decision Tree Regression: Total Sales & Operating Profit Predictions vs. Actual Values (Monthly Data)')

In [256]:
#Define the scoring function to be used in GridSearchCV
mse = make_scorer(mean_squared_error, greater_is_better=False)

#Find optimal parameters using GridSearchCV
dtr_grid = {'max_depth': [None, 5, 10, 15, 20],
            'min_samples_split': [2, 4, 6, 8, 10],
            'min_samples_leaf': [1, 2, 3, 4, 5],
            'max_features': ['auto', 'sqrt', 'log2']}

dtr_CV = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=dtr_grid, scoring=mse, n_jobs=4)
dtr_CV.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [257]:
dtr_CV.best_params_

{'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 4}

In [258]:
#Use the optimal parameters to create a new model
opt_dtr_model = DecisionTreeRegressor(max_depth=20, max_features='sqrt', min_samples_leaf=4, min_samples_split=4, random_state=0)

#Fit model to training data
opt_dtr_model.fit(X_train, y_train)

#Make predictions using the testing data
opt_dtr_predictions = opt_dtr_model.predict(X_test)

In [259]:
#Accuracy of the model
opt_dtr_mse = mean_squared_error(y_test, opt_dtr_predictions)
opt_dtr_rmse = opt_dtr_mse**.5

print(f'Random Forest Regressor MSE: {opt_dtr_mse}')
print(f'Random Forest Regressor RMSE: {opt_dtr_rmse}')

Random Forest Regressor MSE: 2170272314088.687
Random Forest Regressor RMSE: 1473184.412790431


In [260]:
#For loop to find the optimal parameters
opt_dtr_rmse_loop = 1473184

for d in dtr_grid['max_depth']:
    for ms in dtr_grid['min_samples_split']:
        for ml in dtr_grid['min_samples_leaf']:
            for mf in dtr_grid['max_features']:
                opt_dtr_model_loop = DecisionTreeRegressor(max_depth=d, max_features=mf, min_samples_leaf=ml, min_samples_split=ms, random_state=0)
                opt_dtr_model_loop.fit(X_train, y_train)
                opt_dtr_predictions_loop = opt_dtr_model_loop.predict(X_test)
                opt_dtr_mse_loop = mean_squared_error(y_test, opt_dtr_predictions_loop)
                opt_dtr_rmse_in_loop = opt_dtr_mse_loop**.5
                
                #If the RMSE is lower than the previous RMSE, then update
                if opt_dtr_rmse_in_loop < opt_dtr_rmse_loop:
                    opt_dtr_rmse_loop = opt_dtr_rmse_in_loop
                    print(f'RMSE: {opt_dtr_rmse_loop}')
                    print(f'Optimal parameters: max_depth={d}, min_samples_split={ms}, min_samples_leaf={ml}, max_features={mf}')

RMSE: 893761.3668900668
Optimal parameters: max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=auto
RMSE: 707192.4975140383
Optimal parameters: max_depth=None, min_samples_split=2, min_samples_leaf=4, max_features=auto
RMSE: 618378.9143487753
Optimal parameters: max_depth=None, min_samples_split=2, min_samples_leaf=5, max_features=auto


In [261]:
#Use the optimal parameters to create a new model
opt_dtr_model = DecisionTreeRegressor(max_depth=None, max_features='auto', min_samples_leaf=5, min_samples_split=2, random_state=0)

#Fit model to training data
opt_dtr_model.fit(X_train, y_train)

#Make predictions using the testing data
opt_dtr_predictions = opt_dtr_model.predict(X_test)

In [262]:
#Accuracy of the model
opt_dtr_mse = mean_squared_error(y_test, opt_dtr_predictions)
opt_dtr_rmse = opt_dtr_mse**.5

print(f'Random Forest Regressor MSE: {opt_dtr_mse}')
print(f'Random Forest Regressor RMSE: {opt_dtr_rmse}')

Random Forest Regressor MSE: 382392481711.1699
Random Forest Regressor RMSE: 618378.9143487753


In [264]:
#Create a dataframe of the predictions
opt_dtr_predictions_df = pd.DataFrame(opt_dtr_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_test.index)
opt_dtr_predictions_df['Actual_Total_Sales'], opt_dtr_predictions_df['Actual_Operating_Profit'] = y_test['Total_sales'], y_test['Operating_profit']
opt_dtr_predictions_df = opt_dtr_predictions_df.sort_index()
opt_dtr_predictions_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q1,2096273.25,7.749617e+05,2312746.0,883774.16
2020:Q2_3,2096273.25,7.749617e+05,1084194.0,307046.16
2020:Q3_2,2096273.25,7.749617e+05,2641630.0,1042648.29
2020:Q4,2096273.25,7.749617e+05,1428569.0,582342.96
2021:Q2,7213281.00,2.800046e+06,6498339.0,2644237.72
2021:Q4_2,9100768.80,3.692223e+06,7855390.0,3101992.69


In [265]:
#Plot the predictions vs. the actual values
px.line(opt_dtr_predictions_df, x=opt_dtr_predictions_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Optimized Decision Tree Regression: Total Sales & Operating Profit Predictions vs. Actual Values (Monthly Data)')

## XGBoost Regression

In [266]:
#Create model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, n_jobs=4)

#Fit model to training data
xgb_model.fit(X_train, y_train)

#Make predictions using the testing data
xgb_predictions = xgb_model.predict(X_test)

In [267]:
#Accuracy of the model
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_rmse = xgb_mse**.5

print(f'XGBoost Regressor MSE: {xgb_mse}')
print(f'XGBoost Regressor RMSE: {xgb_rmse}')

XGBoost Regressor MSE: 798805151256.7671
XGBoost Regressor RMSE: 893759.0006577652


In [268]:
#Create a dataframe of the predictions
xgb_predictions_df = pd.DataFrame(xgb_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_test.index)
xgb_predictions_df['Actual_Total_Sales'], xgb_predictions_df['Actual_Operating_Profit'] = y_test['Total_sales'], y_test['Operating_profit']
xgb_predictions_df = xgb_predictions_df.sort_index()
xgb_predictions_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q1,2307503.750,8.380574e+05,2312746.0,883774.16
2020:Q2_3,2678921.000,1.038038e+06,1084194.0,307046.16
2020:Q3_2,2275040.000,8.286941e+05,2641630.0,1042648.29
2020:Q4,1123624.625,3.950558e+05,1428569.0,582342.96
2021:Q2,8647948.000,3.443152e+06,6498339.0,2644237.72
2021:Q4_2,8723153.000,3.532938e+06,7855390.0,3101992.69


In [269]:
#Plot the predictions vs. the actual values
px.line(xgb_predictions_df, x=xgb_predictions_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='XGBoost Regression: Total Sales & Operating Profit Predictions vs. Actual Values (Monthly Data)')

In [272]:
#Define the scoring function to be used in GridSearchCV
mse = make_scorer(mean_squared_error, greater_is_better=False)

#Find optimal parameters using GridSearchCV
xgb_grid = {'max_depth': [None, 5, 10, 15, 20],
        'min_child_weight': [1, 2, 3, 4, 5],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_alpha': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0, 0.25, 0.5, 1.0],
        'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1.0]}

#Create the GridSearchCV object
xgb_CV = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, n_jobs=4), 
                      param_grid=xgb_grid, scoring=mse, n_jobs=4)
xgb_CV.fit(X_train, y_train)

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.05, max_bi...
                                    n_estimators=1000, n_jobs=4,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, ...),
             n_jobs=4,
             param_grid={'gamma': [0, 0.25, 0.5, 1.0],
                         '

In [273]:
xgb_CV.best_params_

{'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': None,
 'min_child_weight': 5,
 'reg_alpha': 1.0,
 'reg_lambda': 1.0}

In [274]:
#Use the optimal parameters to create a new model
opt_xgb_model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.1, 
                                 max_depth=None, min_child_weight=5, gamma=0, reg_alpha=1.0, reg_lambda=1.0)

#Fit model to training data
opt_xgb_model.fit(X_train, y_train)

#Make predictions using the testing data
opt_xgb_predictions = opt_xgb_model.predict(X_test)

In [275]:
#Accuracy of the model
opt_xgb_mse = mean_squared_error(y_test, opt_xgb_predictions)
opt_xgb_rmse = opt_xgb_mse**.5

print(f'XGBoost Regressor MSE: {opt_xgb_mse}')
print(f'XGBoost Regressor RMSE: {opt_xgb_rmse}')

XGBoost Regressor MSE: 866637837497.1515
XGBoost Regressor RMSE: 930933.852374674


In [277]:
#For loop to find the optimal parameters
opt_xgb_rmse_loop = 930933

for lr in xgb_grid['learning_rate']:
    for md in xgb_grid['max_depth']:
        for mcw in xgb_grid['min_child_weight']:
            for g in xgb_grid['gamma']:
                for ra in xgb_grid['reg_alpha']:
                    for rl in xgb_grid['reg_lambda']:
                        xgb_model_loop = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=4, learning_rate=lr, 
                                                          max_depth=md, min_child_weight=mcw, gamma=g, reg_alpha=ra, reg_lambda=rl)

                        xgb_model_loop.fit(X_train, y_train)
                        xgb_predictions_loop = xgb_model_loop.predict(X_test)
                        xgb_mse_loop = mean_squared_error(y_test, xgb_predictions_loop)
                        xgb_rmse_in_loop = xgb_mse_loop**.5
                        
                        #If the RMSE is lower than the previous RMSE, then update
                        if xgb_rmse_in_loop < opt_xgb_rmse_loop:
                            opt_xgb_rmse_loop = xgb_rmse_in_loop
                            print(f'RMSE: {opt_xgb_rmse_loop}')
                            print(f'Optimal Parameters: learning_rate={lr}, max_depth={md}, min_child_weight={mcw}, gamma={g}, reg_alpha={ra}, reg_lambda={rl}')

RMSE: 924809.4220717252
Optimal Parameters: learning_rate=0.01, max_depth=None, min_child_weight=1, gamma=0, reg_alpha=0, reg_lambda=0.25
RMSE: 908493.094105406
Optimal Parameters: learning_rate=0.01, max_depth=None, min_child_weight=3, gamma=0, reg_alpha=0, reg_lambda=0
RMSE: 875128.2427081533
Optimal Parameters: learning_rate=0.05, max_depth=None, min_child_weight=1, gamma=0, reg_alpha=0, reg_lambda=0
RMSE: 870906.8821407054
Optimal Parameters: learning_rate=0.05, max_depth=None, min_child_weight=1, gamma=0, reg_alpha=0, reg_lambda=0.25
RMSE: 867095.05377015
Optimal Parameters: learning_rate=0.05, max_depth=None, min_child_weight=1, gamma=0, reg_alpha=0, reg_lambda=0.5
RMSE: 860717.6654119646
Optimal Parameters: learning_rate=0.05, max_depth=None, min_child_weight=1, gamma=0, reg_alpha=0, reg_lambda=1.0
RMSE: 860717.6476760523
Optimal Parameters: learning_rate=0.05, max_depth=None, min_child_weight=1, gamma=0, reg_alpha=0.25, reg_lambda=1.0
RMSE: 860717.628992574
Optimal Parameters: 

In [278]:
#Create a XGBoost Regressor model with the optimal parameters
opt_xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=4, learning_rate=0.05, max_depth=None, 
                                 min_child_weight=1, gamma=0, reg_alpha=1.0, reg_lambda=1.0)

#Fit model to training data
opt_xgb_model.fit(X_train, y_train)

#Make predictions using the testing data
opt_xgb_predictions = opt_xgb_model.predict(X_test)

In [279]:
#Calculate the mean squared error and root mean squared error as performance metrics
opt_xgb_mse = mean_squared_error(y_test, opt_xgb_predictions)
opt_xgb_rmse = opt_xgb_mse**.5

print(f'XGBoost Regressor MSE: {opt_xgb_mse}')
print(f'XGBoost Regressor RMSE: {opt_xgb_rmse}')

XGBoost Regressor MSE: 740834824917.4469
XGBoost Regressor RMSE: 860717.6220558325


In [280]:
#Create a dataframe of the predictions
opt_xgb_predictions_df = pd.DataFrame(opt_xgb_predictions, columns=['Predicted_Total_Sales', 'Predicted_Operating_Profit'], index=y_test.index)
opt_xgb_predictions_df['Actual_Total_Sales'], opt_xgb_predictions_df['Actual_Operating_Profit'] = y_test['Total_sales'], y_test['Operating_profit']
opt_xgb_predictions_df = opt_xgb_predictions_df.sort_index()
opt_xgb_predictions_df

,Predicted_Total_Sales,Predicted_Operating_Profit,Actual_Total_Sales,Actual_Operating_Profit
2020:Q1,2277057.500,8.258546e+05,2312746.0,883774.16
2020:Q2_3,2642997.000,1.023910e+06,1084194.0,307046.16
2020:Q3_2,2258448.250,8.221252e+05,2641630.0,1042648.29
2020:Q4,1112905.125,3.907887e+05,1428569.0,582342.96
2021:Q2,8564761.000,3.409284e+06,6498339.0,2644237.72
2021:Q4_2,8629663.000,3.495616e+06,7855390.0,3101992.69


In [281]:
#Plot the predictions vs. the actual values
px.line(opt_xgb_predictions_df, x=opt_xgb_predictions_df.index, y=['Predicted_Total_Sales', 'Actual_Total_Sales', 'Predicted_Operating_Profit', 'Actual_Operating_Profit'], 
        title='Optimized XGBoost Regression: Total Sales & Operating Profit Predictions vs. Actual Values (Monthly Data)')